### Scraping

In [62]:
#### Importação das bibliotecas necessárias

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import re
import time, random
import json


In [63]:
#### Configurações do webdriver

driver = webdriver.Chrome()
driver.implicitly_wait(10)

In [3]:
#### Acesso à página do Senado

driver.get('http://www.senado.leg.br')

###### - Operações manuais na janela do webdriver: pesquisar por medidas provisórias apresentadas entre 01/01/2014 e 04/07/2024

In [27]:
#### Captura dos links de cada uma das MPs pesquisadas
links = []
counter = 0
while True:
    time.sleep(1 + random.random())
    resultados = driver.find_elements(By.PARTIAL_LINK_TEXT, "MPV")
    for resultado in resultados:
        dict = {}
        dict['URL'] = resultado.get_attribute('href')
        dict['Scraping'] = False
        links.append(dict)
        time.sleep(1 + random.random())
    next_button = driver.find_element(By.PARTIAL_LINK_TEXT, "Próxima")
    time.sleep(1 + random.random())
    try:
        next_button.click()
        counter += 1
        print(f"Página {counter} concluída.")
    except NoSuchElementException:
        print(f"Pesquisa por links concluída. Verificadas {counter} páginas.")

#### Salva links no formato json
with open('json/links.json', 'w', encoding='utf-8') as arquivo_json:
    json.dump(links, arquivo_json, ensure_ascii=False, indent=4)


Página 1 concluída.
Página 2 concluída.
Página 3 concluída.
Página 4 concluída.
Página 5 concluída.
Página 6 concluída.
Página 7 concluída.
Página 8 concluída.
Página 9 concluída.
Página 10 concluída.
Página 11 concluída.
Página 12 concluída.
Página 13 concluída.


KeyboardInterrupt: 

In [64]:
#### Faz scraping das MPs de cada link
from src import scp
with open('json/links.json', 'r', encoding='utf-8') as arquivo_json: # Carrega links salvos
    links = json.load(arquivo_json)
dicts = []
counter = 0
for link in links:
    counter += 1
    print(f"Processando link {counter}/{len(links)}")
    if link['Scraping'] != True:
        dicts.append(scp.scraping(link, driver, random, time, By, re, NoSuchElementException))
        link['Scraping'] = True
        with open('json/mps.json', 'w') as arquivo_json:
            json.dump(dicts, arquivo_json, ensure_ascii=False, indent=4)
        with open('json/links.json', 'w') as arquivo_json:
            json.dump(links, arquivo_json, ensure_ascii=False, indent=4)
        print("Feito.")
    else:
        print(f"Link {counter}/{len(links)} já processado.")
        continue
print("Operação concluída.")

In [65]:
len(links) == len(dicts)

True

#### Análise

In [66]:
# Importa biblioteca pandas
import pandas as pd

In [67]:
# Cria dataframe de MPs
mps = pd.read_json("json/mps.json")

In [68]:
# Exibe colunas
mps.columns

Index(['Proposição', 'URL', 'Ementa', 'Indexação', 'Norma gerada',
       'Audiências públicas realizadas', 'Quantidade de emendas'],
      dtype='object')

In [72]:
# Cria uma nova coluna 'Número' extraindo o ano da coluna 'Proposição'
mps['Número'] = mps['Proposição'].str.extract(r'(\d{4})')
mps['Ano'] = mps['Proposição'].str[-4:]


In [71]:
# Filtra as MPs em que o número de audiências públicas é superior a zero
mps_com_audiencias = mps[mps['Audiências públicas realizadas'] > 0]
# Seleciona apenas as colunas de interesse
resultado = mps_com_audiencias[['Proposição', 'Número', 'Ano', 'Audiências públicas realizadas', 'URL']]
# Mostra o resultado
resultado

,Proposição,Número,Ano,Audiências públicas realizadas,URL
36,"Medida Provisória n° 1202, de 2023",1202,2023,1,https://www.congressonacional.leg.br/materias/...
64,"Medida Provisória n° 1174, de 2023",1174,2023,2,https://www.congressonacional.leg.br/materias/...
65,"Medida Provisória n° 1173, de 2023",1173,2023,2,https://www.congressonacional.leg.br/materias/...
66,"Medida Provisória n° 1172, de 2023",1172,2023,1,https://www.congressonacional.leg.br/materias/...
71,"Medida Provisória n° 1167, de 2023",1167,2023,1,https://www.congressonacional.leg.br/materias/...
73,"Medida Provisória n° 1165, de 2023",1165,2023,4,https://www.congressonacional.leg.br/materias/...
74,"Medida Provisória n° 1164, de 2023",1164,2023,2,https://www.congressonacional.leg.br/materias/...
76,"Medida Provisória n° 1162, de 2023",1162,2023,5,https://www.congressonacional.leg.br/materias/...
84,"Medida Provisória n° 1154, de 2023",1154,2023,1,https://www.congressonacional.leg.br/materias/...
328,"Medida Provisória n° 910, de 2019",2019,2019,4,https://www.congressonacional.leg.br/materias/...


In [74]:
len(resultado)/len(links)

0.04326123128119801